# Stats analysis
This notebook read from dumped scores through different approach and complete stats tests

## import 

In [1]:
import joblib
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.weightstats import DescrStatsW
import numpy as np
import pandas as pd
from ALSampler import summarize
from itertools import chain

In [2]:
m_scores=joblib.load(r'..\data\vb_sampling\ner_crf_scores.joblib')

In [4]:
r_scores=joblib.load(r'..\data\vb_sampling\ner_rand_scores.joblib')

In [5]:
vmax_scores=joblib.load(r'..\data\vb_sampling\ner_vbmax_scores.joblib')

In [6]:
vmin_scores=joblib.load(r'..\data\vb_sampling\ner_vb_scores.joblib')

In [8]:
name1='m'
name2='v'
measures=['precision','recall','f1'] 
cols=[b+"_"+a for b in measures for a in [name1, name2,'pv'] ]
results={col:[] for col in cols}

In [9]:
results

{'precision_m': [],
 'precision_v': [],
 'precision_pv': [],
 'recall_m': [],
 'recall_v': [],
 'recall_pv': [],
 'f1_m': [],
 'f1_v': [],
 'f1_pv': []}

In [10]:
def compute_stats(scores1, scores2, name1, name2, runs=3, rounds=9):
    measures=['precision','recall','f1'] 
    cols=[b+"_"+a for b in measures for a in [name1, name2,'pv'] ]
    results={col:[] for col in cols}
    for ri in range(rounds):
        single_round_scores={name1:{}, name2:{}}
        for m in measures:             
            # print('s1',len(scores1), len(scores1[0]), len(scores1[0][0][m]))
            # print('s2',len(scores2), len(scores2[0]), len(scores2[0][0][m]))
            single_round_scores[name1][m]=list(chain.from_iterable([scores1[u][ri][m] for u in range(runs)]))
            single_round_scores[name2][m]=list(chain.from_iterable([scores2[u][ri][m] for u in range(runs)]))
            results[m+'_'+name1].append(np.mean(single_round_scores[name1][m]))
            results[m+'_'+name2].append(np.mean(single_round_scores[name2][m]))
            ttres=ttest_ind(single_round_scores[name1][m], single_round_scores[name2][m], usevar='unequal')
            # ttres=DescrStatsW.ttest_mean(np.array(single_round_scores[name1][m])-np.array(single_round_scores[name2][m]))
            results[m+'_pv'].append(ttres[1])
    return pd.DataFrame(results)

In [11]:
list(range(3))

[0, 1, 2]

In [12]:
results=compute_stats(m_scores, vmin_scores, 'm', 'vm')

In [13]:
results

,precision_m,precision_vm,precision_pv,recall_m,recall_vm,recall_pv,f1_m,f1_vm,f1_pv
0,0.943678,0.943678,1.000000e+00,0.779015,0.779015,1.000000e+00,0.853472,0.853472,1.000000e+00
1,0.945390,0.940654,1.013302e-273,0.809297,0.812435,9.797694e-55,0.872063,0.871853,1.127057e-01
2,0.945161,0.939739,2.158632e-318,0.827161,0.825684,6.611396e-16,0.882230,0.879025,1.244450e-114
3,0.946089,0.943416,1.578406e-120,0.837118,0.835087,3.536611e-29,0.888271,0.885950,7.250735e-69
4,0.944826,0.944359,2.435177e-06,0.844373,0.844404,8.524970e-01,0.891777,0.891587,1.040886e-01
5,0.944404,0.941879,2.847126e-115,0.852707,0.852372,3.433953e-02,0.896214,0.894891,9.470369e-31
6,0.947097,0.945456,7.115859e-61,0.852947,0.852881,6.728997e-01,0.897558,0.896784,2.310431e-12
7,0.945032,0.945230,4.236964e-02,0.857179,0.855689,5.250165e-21,0.898963,0.898232,4.600355e-11
8,0.948213,0.944960,5.344984e-179,0.857083,0.859023,3.902987e-34,0.900346,0.899942,1.831363e-04


In [14]:
results=compute_stats(m_scores, r_scores, 'm', 'r')
results

,precision_m,precision_r,precision_pv,recall_m,recall_r,recall_pv,f1_m,f1_r,f1_pv
0,0.943678,0.943678,1.000000e+00,0.779015,0.779015,1.000000e+00,0.853472,0.853472,1.000000e+00
1,0.945390,0.943385,2.684113e-76,0.809297,0.808786,6.868865e-03,0.872063,0.870912,2.911611e-18
2,0.945161,0.943099,1.425846e-81,0.827161,0.824602,5.371956e-42,0.882230,0.879876,5.221634e-69
3,0.946089,0.943409,2.537116e-124,0.837118,0.833367,1.806493e-84,0.888271,0.884978,6.671025e-124
4,0.944826,0.943532,6.437076e-37,0.844373,0.842700,2.415489e-22,0.891777,0.890268,1.405758e-35
5,0.944404,0.945672,2.088609e-36,0.852707,0.844814,1.791163e-286,0.896214,0.892400,1.298033e-176
6,0.947097,0.946011,3.384115e-28,0.852947,0.849827,1.806002e-75,0.897558,0.895341,3.045752e-79
7,0.945032,0.944626,4.472983e-05,0.857179,0.854204,2.739306e-70,0.898963,0.897140,1.277517e-55
8,0.948213,0.945657,6.360340e-125,0.857083,0.858068,3.543934e-10,0.900346,0.899734,1.909291e-08


In [15]:
results=compute_stats(r_scores, vmin_scores, 'r', 'v')
results

,precision_r,precision_v,precision_pv,recall_r,recall_v,recall_pv,f1_r,f1_v,f1_pv
0,0.943678,0.943678,1.000000e+00,0.779015,0.779015,1.000000e+00,0.853472,0.853472,1.000000e+00
1,0.943385,0.940654,7.327605e-125,0.808786,0.812435,3.349254e-73,0.870912,0.871853,1.064773e-12
2,0.943099,0.939739,9.928201e-171,0.824602,0.825684,1.096933e-09,0.879876,0.879025,6.284518e-12
3,0.943409,0.943416,9.463550e-01,0.833367,0.835087,7.281335e-22,0.884978,0.885950,1.046109e-14
4,0.943532,0.944359,6.586847e-17,0.842700,0.844404,4.282315e-24,0.890268,0.891587,2.932090e-28
5,0.945672,0.941879,6.761710e-213,0.844814,0.852372,2.329888e-271,0.892400,0.894891,4.158296e-89
6,0.946011,0.945456,6.144719e-09,0.849827,0.852881,8.862480e-73,0.895341,0.896784,2.775203e-37
7,0.944626,0.945230,1.269888e-09,0.854204,0.855689,3.780169e-20,0.897140,0.898232,8.616012e-22
8,0.945657,0.944960,5.409437e-13,0.858068,0.859023,8.611488e-10,0.899734,0.899942,5.198638e-02
